In [78]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.callbacks import TensorBoard
from sklearn.utils import class_weight
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from multiprocessing import  Pool
from tqdm.notebook import tqdm
import nltk
from keras.layers import Dense, Dropout
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from time import time
print("open dataset")
train = pd.read_csv("../dataset/goodreads_train.csv")

open dataset


In [79]:
train_prepro = pd.DataFrame(data=np.load(file = "C:/Users/alhus/PycharmProjects/goodBook/prepro_train_archive_PN_less.npy", allow_pickle=True), columns=['review_text'])['review_text']
# test = pd.read_csv("dataset/goodreads_test.csv")

In [80]:
train_prepro

0         this special book . it started slow first thir...
1         recommended . free : http : //www.audible.com/...
2         a fun , fast paced science fiction thriller . ...
3         recommended reading understand going middle am...
4         i really enjoyed book , lot recommend . it dra...
                                ...                        
899995    3.5 star . popular author agent want character...
899996    this quick read . i read lot book recently one...
899997    * spoiler alert * 3.5 star . this book sweet i...
899998    * spoiler alert another fun read ! 's new assi...
899999    * spoiler alert * 3.5 star i liked ! the story...
Name: review_text, Length: 900000, dtype: object

In [81]:
train['review_text'] = train_prepro

In [82]:
train['review_text']

0         this special book . it started slow first thir...
1         recommended . free : http : //www.audible.com/...
2         a fun , fast paced science fiction thriller . ...
3         recommended reading understand going middle am...
4         i really enjoyed book , lot recommend . it dra...
                                ...                        
899995    3.5 star . popular author agent want character...
899996    this quick read . i read lot book recently one...
899997    * spoiler alert * 3.5 star . this book sweet i...
899998    * spoiler alert another fun read ! 's new assi...
899999    * spoiler alert * 3.5 star i liked ! the story...
Name: review_text, Length: 900000, dtype: object

In [83]:
print("0  ",train[train['rating'] == 0]["rating"].count())
print("1  ", train[train['rating'] == 1]["rating"].count())
print("2  ", train[train['rating'] == 2]["rating"].count())
print("3  ", train[train['rating'] == 3]["rating"].count())
print("4  ", train[train['rating'] == 4]["rating"].count())
print("5  ", train[train['rating'] == 5]["rating"].count())
#train_3_5 = train[train['rating'] >= 3]
#train['rating'] = train['rating'].apply(lambda x: x-3)

0   30988
1   28718
2   72627
3   188972
4   313688
5   265007


In [84]:
#train = train[train['rating'] >= 3]

In [85]:
rating = keras.utils.to_categorical(train['rating'], num_classes=6)

In [86]:
rating = rating.astype(int)

In [87]:
type(rating)

numpy.ndarray

In [88]:
mylen = len(np.load('C:/Users/alhus/PycharmProjects/goodBook/voc_lemm_without_NP.npy'))

In [89]:
mylen

391582

In [90]:
inputs = keras.Input(shape=(1,), dtype=tf.string)
"""inputs2 = keras.Input(shape=(1), dtype=tf.int64)
inputs3 = keras.Input(shape=(1), dtype=tf.int64)"""

vectorize_layer = keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=1400,
    vocabulary=np.load('C:/Users/alhus/PycharmProjects/goodBook/voc_lemm_without_NP.npy')
)(inputs)

In [91]:
vectorize_layer.shape

TensorShape([None, 1400])

In [92]:
#conc = keras.layers.concatenate([vectorize_layer])

embedding_layer = tf.keras.layers.Embedding(input_dim=mylen+1,output_dim=64,mask_zero=True)(vectorize_layer)
flattened_layer = keras.layers.Flatten()(embedding_layer)
layer1 = keras.layers.Dense(64, activation=tf.keras.activations.tanh)(flattened_layer)
layer2 = keras.layers.Dense(32, activation=tf.keras.activations.tanh)(layer1)
layer3 = keras.layers.Dense(32, activation=tf.keras.activations.tanh)(layer2)
layer4 = keras.layers.Dense(16, activation=tf.keras.activations.tanh)(layer3)


outputs = keras.layers.Dense(6, activation=tf.keras.activations.softmax)(layer4)

In [93]:
outputs.shape

TensorShape([None, 6])

In [94]:
model = keras.Model(inputs=[inputs], outputs=outputs, name="mnist_model")
tensorboard = TensorBoard(log_dir="../logs/relu".format(time()))

In [95]:
#learning_rates = [0.01, 0.001, 0.000001, 0.0000001]

In [96]:
#for learning_rate in learning_rates:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy]
              )

In [97]:
x_val = train['review_text'][-10000:]
y_val = rating[-10000:]
x_train = train['review_text'][:-10000]
y_train = rating[:-10000]
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes= np.unique(train['rating']), y = train['rating'])

In [98]:
weight = {i : class_weights[i] for i in range(6)}

In [99]:
weight

{0: 4.8405834516587065,
 1: 5.223204958562574,
 2: 2.0653475980007436,
 3: 0.7937683889676778,
 4: 0.4781821427660605,
 5: 0.5660227843038108}

In [100]:
model.fit(train['review_text'], rating, epochs=10,
                  callbacks=[
                      tf.keras.callbacks.TensorBoard(log_dir="../logs/relu"),
                  ],
                  batch_size=1000, validation_data=[x_val, y_val], shuffle= True, class_weight= weight
                  )

Epoch 1/10
900/900 [==============================] - 57s 63ms/step - loss: 1.6377 - categorical_accuracy: 0.2504 - val_loss: 1.3031 - val_categorical_accuracy: 0.4549
Epoch 2/10
900/900 [==============================] - 55s 62ms/step - loss: 1.2525 - categorical_accuracy: 0.4747 - val_loss: 1.1553 - val_categorical_accuracy: 0.5246
Epoch 3/10
900/900 [==============================] - 54s 60ms/step - loss: 1.1131 - categorical_accuracy: 0.5243 - val_loss: 1.0109 - val_categorical_accuracy: 0.6003
Epoch 4/10
900/900 [==============================] - 55s 61ms/step - loss: 0.9301 - categorical_accuracy: 0.5896 - val_loss: 0.8940 - val_categorical_accuracy: 0.6598
Epoch 5/10
900/900 [==============================] - 55s 61ms/step - loss: 0.7458 - categorical_accuracy: 0.6594 - val_loss: 0.7546 - val_categorical_accuracy: 0.7201
Epoch 6/10
900/900 [==============================] - 54s 60ms/step - loss: 0.6045 - categorical_accuracy: 0.7160 - val_loss: 0.6554 - val_categorical_accuracy:

In [101]:
model.save("../models_trained/PMC_embedding_model_10_class_weights")

INFO:tensorflow:Assets written to: ../models_trained/PMC_embedding_model_10_class_weights\assets


In [102]:
test = pd.read_csv("../dataset/goodreads_test.csv")

In [103]:
test_prepro = pd.DataFrame(data=np.load(file="C:/Users/alhus/PycharmProjects/goodBook/prepro_test_archive_PN_less.npy", allow_pickle=True), columns=['review_text'])['review_text']
test['review_text'] = test_prepro

In [104]:
test

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,b9450d1c1f97f891c392b1105959b56e,7092507,5c4df7e70e9b438c761f07a4620ccb7c,* spoiler alert this definitely one favorite a...,Sat Nov 10 06:06:13 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sat Nov 10 00:00:00 -0800 2012,1,0
1,b9450d1c1f97f891c392b1105959b56e,5576654,8eaeaf13213eeb16ad879a2a2591bbe5,* spoiler alert `` you drink . '' i 'm huge fa...,Fri Nov 09 21:55:16 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,1,0
2,b9450d1c1f97f891c392b1105959b56e,15754052,dce649b733c153ba5363a0413cac988f,one favorite character under i 'm happy read s...,Fri Nov 09 00:25:50 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,0,0
3,b9450d1c1f97f891c392b1105959b56e,17020,8a46df0bb997269d6834f9437a4b0a77,* spoiler alert if feel like travelling n't mo...,Thu Nov 01 00:28:39 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Thu Nov 01 00:00:00 -0700 2012,0,0
4,b9450d1c1f97f891c392b1105959b56e,12551082,d11d3091e22f1cf3cb865598de197599,3.5 star i read enjoyed first two novel series...,Thu Oct 18 00:57:00 -0700 2012,Mon Apr 01 23:00:51 -0700 2013,Sat Mar 30 00:00:00 -0700 2013,Fri Mar 29 00:00:00 -0700 2013,0,0
...,...,...,...,...,...,...,...,...,...,...
478028,35cef391b171b4fca45771e508028212,15745950,0e1db3d4b04256f9660f5d276ddf1314,n't wait ' before ... after ...,Sun Aug 05 10:26:12 -0700 2012,Tue Apr 16 17:24:00 -0700 2013,Tue Apr 16 00:00:00 -0700 2013,NaN,0,0
478029,35cef391b171b4fca45771e508028212,10861195,0b7f352e58caf0fd1f961e98ef04e89c,to-read shelf forever . update i 've finished ...,Tue Jul 10 23:31:00 -0700 2012,Fri Dec 28 20:05:51 -0800 2012,NaN,NaN,0,0
478030,35cef391b171b4fca45771e508028212,6131164,9b19eff33ddb14e9e68fca2e90379e46,the last book left wanting . i need happy endi...,Tue Jul 10 19:45:17 -0700 2012,Mon Mar 25 18:41:51 -0700 2013,Tue Mar 19 00:00:00 -0700 2013,NaN,0,0
478031,35cef391b171b4fca45771e508028212,10025305,8be463fed78f0da63e964706f710332b,things heating second novel devices . will beg...,Thu Jul 05 19:19:30 -0700 2012,Thu Jan 24 16:24:54 -0800 2013,Mon Jan 14 00:00:00 -0800 2013,NaN,0,0


In [ ]:
restest = model.predict([test['review_text']])

12840/14939 [========================>.....] - ETA: 10s

In [ ]:
ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
data = np.array(ff)

In [ ]:
test['rating'] = data

In [ ]:
id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])

In [ ]:
df['review_id'] = id
df['rating'] = rating

In [ ]:
df.to_csv('submission_pmc10_embedding_class_weights_model.csv',index=False )